<a href="https://colab.research.google.com/github/park-hoyeon/park-hoyeon.github.io/blob/master/skt_7_03_pytorch_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

In [ ]:
import torchvision.transforms as transforms

In [ ]:
mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5),(1.0))
])

In [ ]:
from torchvision.datasets import MNIST
import requests

download_root = '../chap02/data/MNIST_DATASET'

In [ ]:
train_dataset = MNIST(download_root, transform=mnist_transform, train = True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train = False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train = False, download=True)

In [ ]:
import torch.nn as nn
model = nn.Linear(in_features=1, out_features=1, bias=True)

In [ ]:
class MLP(nn.Module):
    def __init__(self, inputs):
        super(MLP, self).__init__()
        self.layer = Linear(inputs, 1)
        self.activation = Sigmoid()

    def forward(self, X):
        X = self.layer(X)
        X = self.activation(X)
        return X

In [ ]:
import torch.nn as nn
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=64, kernel_size=5),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2))

        self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels=64, out_channels=30, kernel_size=5),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2))

        self.layer3 = nn.Sequential(
        nn.Linear(in_features=30*4*4, out_features=10, bias=True),
        nn.ReLU(inplace=True))

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.shape[0], -1)
        x = self.layer3(x)
        return x

model = MLP()

In [ ]:
print(list(model.modules()))

In [ ]:
print(list(model.children()))

In [ ]:
import torch
from torch.optim import optimizer
from torch.utils.data import DataLoader

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda epoch: 0.95 ** epoch)

In [ ]:
dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True) #train_loader

In [ ]:
for epoch in range(1, 100+1):
    for x, y in dataloader:
        optimizer.zero_grad()
        criterion(model(x), y).backward()
        optimizer.step()
    scheduler.step()

In [ ]:
for epoch in range(100):
    yhat = model(x_train)
    loss = criterion(yhat, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# 간단한 실습


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

batch_size = 64
learning_rate = 0.001
num_epochs = 5

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # 2D 이미지를 1D로 변환
        x = torch.relu(self.fc1(x))  # 활성화 함수
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
model = SimpleNN().to(device='cpu')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 모델 학습 (only train)
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device='cpu'), labels.to(device='cpu')
        optimizer.zero_grad()  # 기울기 초기화
        outputs = model(images)  # 모델 예측
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 가중치 업데이트

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("모델 학습 완료!")

In [ ]:
# 모델 평가
model.eval()  # 평가 모드로 전환
correct = 0
total = 0

with torch.no_grad():  # 기울기 계산 비활성화
    for images, labels in test_loader:  # 테스트 데이터 로더에서 배치 가져오기
        images, labels = images.to(device='cpu'), labels.to(device='cpu')  # 데이터 이동
        outputs = model(images)  # 모델 예측
        _, predicted = torch.max(outputs.data, 1)  # 예측된 클래스
        total += labels.size(0)  # 총 샘플 수
        correct += (predicted == labels).sum().item()  # 올바른 예측 수

accuracy = 100 * correct / total  # 정확도 계산
print(f'Accuracy of the model on the test set: {accuracy:.2f}%')

In [ ]:
# 모델 학습 및 평가 with valid
for epoch in range(num_epochs):
    model.train()  # 학습 모드로 전환
    for images, labels in train_loader:
        images, labels = images.to(device='cpu'), labels.to(device='cpu')
        optimizer.zero_grad()  # 기울기 초기화
        outputs = model(images)  # 모델 예측
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 가중치 업데이트

    # 매 에폭마다 평가
    model.eval()  # 평가 모드로 전환
    correct = 0
    total = 0

    with torch.no_grad():  # 기울기 계산 비활성화
        for images, labels in valid_loader:  # 평가 데이터 로더에서 배치 가져오기
            images, labels = images.to(device='cpu'), labels.to(device='cpu')  # 데이터 이동
            outputs = model(images)  # 모델 예측
            _, predicted = torch.max(outputs.data, 1)  # 예측된 클래스
            total += labels.size(0)  # 총 샘플 수
            correct += (predicted == labels).sum().item()  # 올바른 예측 수

    accuracy = 100 * correct / total  # 정확도 계산
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')

print("모델 학습 완료!")

In [ ]:
!pip install torchmetrics

In [ ]:
import torch
import torchmetrics

preds = torch.randn(10, 5).softmax(dim=-1)
target = torch.randint(5, (10,))

acc = torchmetrics.functional.accuracy(preds, target, task='multiclass', num_classes=5)
print(f'Accuracy: {acc}')

In [ ]:
metric = torchmetrics.Accuracy(task='multiclass', num_classes=5)

n_batches = 10

for i in range(n_batches):
    preds = torch.randn(10, 5).softmax(dim=-1)
    target = torch.randint(5, (10,))

    acc = metric(preds, target)
    print(f'Accuracy on batch {i}: {acc}')

acc = metric.compute()
print(f'Accuracy on all data: {acc}')

# Practice2